In [2]:
import pandas as pd
import numpy as np
import os
import duckdb
import datetime

In [3]:
# dimension table
product_master = pd.read_parquet('../dataset/parquet/PRODUCT_MASTER.parquet')
store_master = pd.read_parquet('../dataset/parquet/STORE_MASTER.parquet')
user_master = pd.read_parquet('../dataset/parquet/USER_MASTER.parquet')
# fact tables
operation_details = pd.read_parquet('../dataset/parquet/FACT_OPERATION_TABLE.parquet')
order_details = pd.read_parquet('../dataset/parquet/FACT_ORDER_TABLE.parquet')

In [4]:
# print shape of each dataframe
print("Product Master Shape:", product_master.shape)
print("Store Master Shape:", store_master.shape)
print("User Master Shape:", user_master.shape)
print("Operation Details Shape:", operation_details.shape)
print("Order Details Shape:", order_details.shape)

Product Master Shape: (1549, 11)
Store Master Shape: (84, 6)
User Master Shape: (592830, 4)
Operation Details Shape: (1430061, 22)
Order Details Shape: (2060463, 31)


In [5]:
product_master.head()

,SKU_ID,SKU_NAME,SKU_BRAND,SKU_CATEGORY,SKU_SUBCATEGORY,SKU_RETURNABLE_CONTAINER,SKU_ALCOHOLIC,SKU_OWN_CATALOG,SKU_CONTAINER,SKU_IS_RETURNABLE,SKU_IS_BUNDLE
0,fbae0786652f83ddc651d0cd36188b40103f33d9b5ff7f...,(2) Beer Budweiser Bottle 343 ml 6 un,Budweiser,Beer,Beer,None,1,1,NRB,false,0
1,b33722a436bb87d876e490d347a15c32cc6a61b61725e4...,Paulaner Weiss Can (269ml) Pack x 6,Paulaner,Beer,Beer,None,1,1,CAN,0,0
2,9c73ed825c15f3587dbaae07e4f01c8fb152cf2d5f0d2c...,(1) Beer Pilsator Can 355 ml 6 un,Pilsator,Beer,Beer,None,1,1,CAN,0,0
3,8c76e6477fec2c1b69a8c5026b885352da383f1d3dc637...,(1) Beer San Miguel Can 355 ml 6 un,San Miguel,Beer,Beer,None,1,1,CAN,false,0
4,8084efed817688b3b61a3a9ee104e0d08476e6b9b0fdf9...,(1) Energy drink Red Bull Can 250 ml 1 un,Red Bull,Drinks,Energy Drink,None,0,0,-,false,0


In [6]:
store_master.head()


,STORE_ID,STORE_NAME,STATUS,CITY,PREFIX,DENOMINATION
0,b4dbd85fdc14f93f1db4a80538a5d3ee3184d606e6d00e...,Nairobi,ACTIVE,MIA,DS,darkstore
1,3a9280a7ef0535508e92e5bbc743b6af83c358a176377d...,Belmopan,ACTIVE,MIA,DS,darkstore
2,2df16f89e221264d08396b08e45fd3a8cd71ececdf947c...,Andorra la Vella,ACTIVE,MIA,DS,darkstore
3,bc853a9baae81a14cc2a279c23f7b9af149588f80b7ffd...,Madrid,ACTIVE,MIA,DS,darkstore
4,af32eedaaea7c5c9cb8fa60c08d1c2a1d878104a054698...,San Jose,ACTIVE,MIA,DS,darkstore


In [7]:
user_master.head()

,REGISTRATION_TIMESTAMP,CREATION_DATE,CUSTOMER_ID,USER_KEY
0,2024-06-20 20:21:37.743,2025-01-01 20:46:17.343,None,None
1,2024-06-08 19:51:37.946,2024-06-09 23:46:18.652,7da798fa1c112e43f69d90871c7f76aa7c075cd82a84da...,bbe65953e402fcaf00af1f5c0f34dcc8f28c3e76b7d0ee...
2,2024-02-16 17:22:29.373,2024-02-28 23:52:35.421,21eecbfa0c075076f049eb5e50bc232d7611a37d6cb1bd...,cc497bbd5a430cc16617f75cd42c687fa1e86f59b2b4c3...
3,2024-05-29 14:40:28.968,2025-03-29 05:55:59.578,None,None
4,2024-06-18 14:27:31.085,2024-06-18 20:52:56.310,None,None


In [8]:
user_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592830 entries, 0 to 592829
Data columns (total 4 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   REGISTRATION_TIMESTAMP  592829 non-null  object
 1   CREATION_DATE           592829 non-null  object
 2   CUSTOMER_ID             242130 non-null  object
 3   USER_KEY                236814 non-null  object
dtypes: object(4)
memory usage: 18.1+ MB


In [9]:
# covert REGISTRATION_TIMESTAMP to datetime AND CREATION_DATE to datetime
user_master['REGISTRATION_TIMESTAMP'] = pd.to_datetime(user_master['REGISTRATION_TIMESTAMP'])
user_master['CREATION_DATE'] = pd.to_datetime(user_master['CREATION_DATE'])

In [10]:
order_details.head()

,ORDER_DATE,ORDER_TIME,ORDER_TIMESTAMP,UNIT_QUANTITY,UNIT_VOLUME,UNIT_LIST_PRICE,UNIT_OFFERED_PRICE,UNIT_CHARGED_PRICE,UNIT_DELIVERY_PRICE,UNIT_DISCOUNT,...,TOTAL_VOLUME,SKU_ID,ORDER_ID,CUSTOMER_ID,COUPON_ID,DISCUNT_TYPE_ID,OPERATOR_ID,CAMPAIGN_ID,DEVICE_ID,STORE_ID
0,2021-12-09,15:00:00,2021-12-09 15:45:46.283,1,NaN,1.51,1.51,1.51,0.0,0.00,...,NaN,1bad6b8cf97131fceab8543e81f7757195fbb1d36b376e...,396870f51da14527f8574debc5a38d31f0639eee43a9a5...,0feac7456da1dc563dd5b320f5f867520b925b401b0c60...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,298be79d248ca0ec4383be30c219a8c9a7b60e6d15ec0b...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,298be79d248ca0ec4383be30c219a8c9a7b60e6d15ec0b...
1,2021-12-30,10:00:00,2021-12-30 10:19:15.242,1,0.000142,6.50,6.50,5.20,0.0,1.30,...,0.000142,41c3b51900da35f4f3810675d7d04ed0d447b27b070988...,f32cbfef6104f537be85682c31df907432c9293d46d83f...,1a38ad305d9d89845bb4a07f1ba01ac8276d8e2c18bacc...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,298be79d248ca0ec4383be30c219a8c9a7b60e6d15ec0b...,560f89b4e57b3d5077def298c5b90607a8f077f028d60b...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,298be79d248ca0ec4383be30c219a8c9a7b60e6d15ec0b...
2,2021-12-12,19:00:00,2021-12-12 19:36:00.358,2,0.013200,35.60,35.60,20.02,0.0,15.58,...,0.026400,af1e47985056a8a1edffdb0770e7e85a419c432547c0c2...,be71e684c855c31d9f327b4450ff0df0b42fe5d30ba29a...,5e9fbcf540e7164b5f8a649a719defcfc53c31ae992418...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,8ebacabc393f66d5c3ba210dca34d542bc4c94b93ccd53...,12ecd42bb988dd9727341621b1f4a3432d4bcc91e4f992...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,8ebacabc393f66d5c3ba210dca34d542bc4c94b93ccd53...
3,2021-12-04,19:00:00,2021-12-04 19:35:00.968,1,0.025200,42.00,42.00,26.91,0.0,15.09,...,0.025200,e4d30fb5a3db1e00334d6b96a1e6e5bffc640a8fddb246...,4fad0bc7c15c826b14995a4985291b08fc4eb1985b25f2...,1d7fbb1c62ae524b36f6f9d5ca1f3cf0092f91241a9a4e...,2b68c6ca4a9c52467143134d9ad2c3d1688a90564cb3c1...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,298be79d248ca0ec4383be30c219a8c9a7b60e6d15ec0b...,8a62029aef1e4faf78f81decf0f914682341944091c7d4...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,298be79d248ca0ec4383be30c219a8c9a7b60e6d15ec0b...
4,2021-12-18,16:00:00,2021-12-18 16:42:19.791,1,0.054900,63.00,63.00,43.90,0.0,19.10,...,0.054900,6f46e0c2d01279c387383fb383a83edb7d70ed113e65c4...,0c5b62c613f1771c17bb0dfe40d0c82ef4c3c58ee4ceab...,f8a8c52328463163394df658afa72468ce9e4890ff05e8...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,492177838841fe53109fc45ec656f71d274b0bbc6df790...,7bfb4b82499a04dc02e531402827ad41498d2928c8ef7c...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,492177838841fe53109fc45ec656f71d274b0bbc6df790...


In [11]:
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2060463 entries, 0 to 2060462
Data columns (total 31 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   ORDER_DATE                    object 
 1   ORDER_TIME                    object 
 2   ORDER_TIMESTAMP               object 
 3   UNIT_QUANTITY                 int64  
 4   UNIT_VOLUME                   float64
 5   UNIT_LIST_PRICE               float64
 6   UNIT_OFFERED_PRICE            float64
 7   UNIT_CHARGED_PRICE            float64
 8   UNIT_DELIVERY_PRICE           float64
 9   UNIT_DISCOUNT                 float64
 10  UNIT_TAX                      float64
 11  UNIT_GROSS                    float64
 12  UNIT_NET                      float64
 13  ORDER_STATE                   object 
 14  ORDER_STATE_FINAL             object 
 15  CUSTOMER_FIRST_ORDER_PERIOD   int64  
 16  CUSTOMER_FIRST_ORDER_YEAR     int64  
 17  ORDER_DAYS_AFTER_FIRST_ORDER  int64  
 18  CUSTOMER_FIRST_ORDER_D

In [12]:
# convert columns to datetime amd time
order_details['ORDER_TIMESTAMP'] = pd.to_datetime(order_details['ORDER_TIMESTAMP'], errors='coerce')
order_details['ORDER_DATE'] = pd.to_datetime(order_details['ORDER_DATE'], errors='coerce')
order_details['ORDER_TIME'] = pd.to_datetime(order_details['ORDER_TIME'], format='%H:%M:%S', errors='coerce').dt.time

In [13]:
order_details.shape

(2060463, 31)

In [14]:
order_details.STORE_ID.unique().shape[0]

126

In [15]:
store_master.STORE_ID.unique().shape[0]

84

In [16]:
# map the STORE_NAME to STORE_ID in order_details
order_details = order_details.merge(
    store_master[['STORE_ID', 'STORE_NAME','PREFIX']],
    on='STORE_ID',
    how='left'
)

In [17]:
order_details.STORE_NAME.unique().shape[0]

85

In [18]:
order_details[order_details['STORE_NAME'].isnull()].shape[0]

18837

In [19]:
# drop the rows with null STORE_NAME
order_details = order_details.dropna(subset=['STORE_NAME'])
# check the unique STORE_NAME again
order_details.STORE_NAME.unique().shape[0]

84

In [20]:
order_details = order_details.merge(
    product_master[[
    'SKU_ID',
    'SKU_NAME',
    'SKU_BRAND',
    'SKU_CATEGORY',
    'SKU_SUBCATEGORY',
    'SKU_RETURNABLE_CONTAINER',
    'SKU_ALCOHOLIC',
    'SKU_OWN_CATALOG',
    'SKU_CONTAINER',
    'SKU_IS_RETURNABLE', 
    'SKU_IS_BUNDLE'
]],
    on='SKU_ID',
    how='left'
)

In [21]:
order_details['SKU_CATEGORY'] = order_details['SKU_CATEGORY'].fillna('Unknown')
order_details['SKU_IS_BUNDLE'] = order_details['SKU_IS_BUNDLE'].fillna(0)

In [22]:
order_details = order_details.merge(
    user_master[['CUSTOMER_ID', 'REGISTRATION_TIMESTAMP', 'CREATION_DATE']],
    on='CUSTOMER_ID',
    how='left'
)

In [23]:
order_details.PAYMENT_METHOD.value_counts()

PAYMENT_METHOD
CASH                   436350
CARD                   414178
E WALLET CASHASPP      403990
CARD ON DELIVERY       285880
SAFE_PAYMENT_ONLINE    209556
CREDIT_CARD_ONLINE     112299
ONLINE_PAYMENTS         98427
E WALLET REVOLUT        80942
QR CODE                     4
Name: count, dtype: int64

In [24]:
# Dictionary to map cleaned PAYMENT_METHOD values
payment_mapping = {
    "CREDIT_CARD_ONLINE": "ONLINE_PAYMENTS",
    "SAFE_PAYMENT_ONLINE": "ONLINE_PAYMENTS",
    "ONLINE_PAYMENTS": "ONLINE_PAYMENTS",
    "CARD ON DELIVERY": "CARD",
    "CARD": "CARD",
    "CASH": "CASH",
    "E WALLET CASHASPP": "E WALLET",
    "E WALLET REVOLUT": "E WALLET",
    "QR CODE": "E WALLET"
}

# Map the values
order_details['PAYMENT_METHOD'] = order_details['PAYMENT_METHOD'].map(payment_mapping)


In [25]:
order_details.PAYMENT_METHOD.value_counts()

PAYMENT_METHOD
CARD               700058
E WALLET           484936
CASH               436350
ONLINE_PAYMENTS    420282
Name: count, dtype: int64

In [26]:
order_details.ORDER_STATE_FINAL.unique()

array(['Valid', 'Invalid', 'Fraud'], dtype=object)

In [27]:
order_details.ORDER_STATE_FINAL.value_counts()

ORDER_STATE_FINAL
Valid      1819117
Invalid     221985
Fraud          524
Name: count, dtype: int64

In [28]:
# drop the rows with Invalid and Fraud ORDER_STATE_FINAL
order_details = order_details[
    ~order_details['ORDER_STATE_FINAL'].isin(['Fraud'])
]
# check the unique ORDER_STATE_FINAL again
order_details.ORDER_STATE_FINAL.unique()

array(['Valid', 'Invalid'], dtype=object)

In [29]:
order_details.SKU_NAME.isna().sum()    

np.int64(140)

In [30]:
# get all the SKU_IDs with null SKU_NAME
null_sku_ids = order_details[order_details['SKU_NAME'].isna()]['SKU_ID'].unique()
null_sku_ids

array(['1bad6b8cf97131fceab8543e81f7757195fbb1d36b376ee994ad1cf17699c464',
       'c968ea6501c846b86cd4047898c1310b767d6c78ccfaed0504192cb9179a206e',
       'f6a697eca0318c15ace5f727c86982ff85db20f34b0a587a0b7919ee40d88b3f'],
      dtype=object)

In [31]:
# drop the rows with null SKU_NAME
order_details = order_details.dropna(subset=['SKU_NAME'])
# check the unique SKU_NAME again
order_details.SKU_NAME.isna().sum()

np.int64(0)

In [32]:
order_details.shape

(2040962, 45)

In [33]:
operation_details.shape

(1430061, 22)

In [34]:
operation_details['ORDER_DATE'] = pd.to_datetime(operation_details['ORDER_DATE'])
operation_details['ORDER_TIME'] = pd.to_datetime(operation_details['ORDER_TIME'], format='%H:%M:%S', errors='coerce')

In [35]:
operation_details = operation_details.merge(
    order_details[['ORDER_ID', 'CUSTOMER_ID']].drop_duplicates(),
    on='ORDER_ID',
    how='left'
)

In [36]:
# CUSTOMER ID SAMPLE 5e9fbcf540e7164b5f8a649a719defcfc53c31ae992418c33793bc2058bd5583
# ORDER ID SAMPLE be71e684c855c31d9f327b4450ff0df0b42fe5d30ba29a21dc11ff736cc0931c

In [37]:
sample_order = duckdb.query("""
SELECT 
    ORDER_ID
    , ORDER_TIMESTAMP
    , UNIT_QUANTITY
    , UNIT_LIST_PRICE
    , UNIT_OFFERED_PRICE
    , UNIT_CHARGED_PRICE
    , UNIT_DELIVERY_PRICE
    , UNIT_DISCOUNT
    , UNIT_GROSS
    , (UNIT_LIST_PRICE - UNIT_OFFERED_PRICE) AS UNIT_ORGANIC_DISCOUNT
    , (UNIT_OFFERED_PRICE - UNIT_CHARGED_PRICE) AS UNIT_EXTRA_DISCOUNT
    , (UNIT_LIST_PRICE - UNIT_OFFERED_PRICE) * UNIT_QUANTITY AS UNIT_ORGANIC_DISCOUNT_TOTAL
    , (UNIT_OFFERED_PRICE - UNIT_CHARGED_PRICE) * UNIT_QUANTITY AS UNIT_EXTRA_DISCOUNT_TOTAL
    , UNIT_NET
FROM order_details
WHERE true
AND CUSTOMER_ID = '5e9fbcf540e7164b5f8a649a719defcfc53c31ae992418c33793bc2058bd5583'
ORDER BY ORDER_TIMESTAMP DESC
""").to_df()
sample_order

,ORDER_ID,ORDER_TIMESTAMP,UNIT_QUANTITY,UNIT_LIST_PRICE,UNIT_OFFERED_PRICE,UNIT_CHARGED_PRICE,UNIT_DELIVERY_PRICE,UNIT_DISCOUNT,UNIT_GROSS,UNIT_ORGANIC_DISCOUNT,UNIT_EXTRA_DISCOUNT,UNIT_ORGANIC_DISCOUNT_TOTAL,UNIT_EXTRA_DISCOUNT_TOTAL,UNIT_NET
0,7443b376d2feddcd3442fa30a9fc21985583de357cb8b1...,2024-12-29 23:01:32.258,1,54.01,41.90,41.90,4.90,0.00,58.91,12.11,0.00,12.11,0.00,46.80
1,c8bcfb78282fb82a4c619030b4f6935f6eb4931fefba72...,2024-12-29 21:19:26.536,1,108.01,94.91,94.91,3.90,0.00,111.91,13.10,0.00,13.10,0.00,98.81
2,094d9ee5570b1c94a9eca44b9d7d2c01c1f90e06db4a0f...,2024-12-29 18:48:24.002,1,108.01,94.91,94.91,2.57,0.00,110.58,13.10,0.00,13.10,0.00,97.48
3,094d9ee5570b1c94a9eca44b9d7d2c01c1f90e06db4a0f...,2024-12-29 18:48:24.002,1,56.40,48.90,48.90,1.33,0.00,57.73,7.50,0.00,7.50,0.00,50.23
4,7102b1654de75cd9cfeb19a143e9cdd433a468f1f91660...,2024-11-21 21:14:58.840,2,33.00,29.90,29.90,2.90,0.00,68.90,3.10,0.00,6.20,0.00,62.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,be71e684c855c31d9f327b4450ff0df0b42fe5d30ba29a...,2021-12-12 19:36:00.358,2,23.60,23.60,13.28,0.00,10.32,47.20,0.00,10.32,0.00,20.64,26.56
116,be71e684c855c31d9f327b4450ff0df0b42fe5d30ba29a...,2021-12-12 19:36:00.358,2,26.00,26.00,14.62,0.00,11.38,52.00,0.00,11.38,0.00,22.76,29.24
117,2eb02c31369dc7f848af5267f3858e5d3573dfedc3ea4a...,2021-11-24 18:53:18.499,1,24.00,24.00,14.31,0.00,9.69,24.00,0.00,9.69,0.00,9.69,14.31
118,2eb02c31369dc7f848af5267f3858e5d3573dfedc3ea4a...,2021-11-24 18:53:18.499,1,26.00,26.00,17.55,0.00,8.45,26.00,0.00,8.45,0.00,8.45,17.55


In [38]:
# Compute discount columns
order_details['UNIT_ORGANIC_DISCOUNT'] = order_details['UNIT_LIST_PRICE'] - order_details['UNIT_OFFERED_PRICE']
order_details['UNIT_EXTRA_DISCOUNT'] = order_details['UNIT_OFFERED_PRICE'] - order_details['UNIT_CHARGED_PRICE']
order_details['UNIT_TOTAL_ORGANIC_DISCOUNT'] = order_details['UNIT_ORGANIC_DISCOUNT'] * order_details['UNIT_QUANTITY']
order_details['UNIT_TOTAL_EXTRA_DISCOUNT'] = order_details['UNIT_EXTRA_DISCOUNT'] * order_details['UNIT_QUANTITY']
order_details['UNIT_TOTAL_DISCOUNTS'] = order_details['UNIT_TOTAL_ORGANIC_DISCOUNT'] + order_details['UNIT_TOTAL_EXTRA_DISCOUNT']

In [39]:
order_details.shape

(2040962, 50)

In [40]:
print(order_details[order_details['ORDER_STATE_FINAL']=='Valid'].shape[0])
print(order_details[order_details['ORDER_STATE_FINAL']=='Invalid'].shape[0])

1818987
221975


In [41]:
valid_orders = order_details[order_details['ORDER_STATE_FINAL']=='Valid']

In [42]:
valid_orders.CUSTOMER_ID.unique().shape[0]

223393

## RFM CALCULATION


In [43]:
# Aggregate per CUSTOMER_ID only for VALID orders
reference_date = order_details['ORDER_DATE'].max()

# RFM & Enhanced Metrics
rfm_df = valid_orders.groupby(['CUSTOMER_ID']).agg(
    Recency=('ORDER_DATE', lambda x: (reference_date - x.max()).days),
    Frequency=('ORDER_ID', 'nunique'),
    Monetary=('UNIT_NET', 'sum'),
    Total_Units=('UNIT_QUANTITY', 'sum'),
    Total_Organic_Discount=('UNIT_TOTAL_ORGANIC_DISCOUNT', 'sum'),
    Total_Extra_Discount=('UNIT_TOTAL_EXTRA_DISCOUNT', 'sum'),
    Total_Discounts=('UNIT_TOTAL_DISCOUNTS', 'sum')
).reset_index()

# rfm_df['Churn_Label'] = (rfm_df['Recency'] > 60).astype(int) 

In [44]:
# Step 1: Aggregate per ORDER_ID and CUSTOMER_ID to get per-order metrics first
order_level = valid_orders.groupby(['CUSTOMER_ID', 'ORDER_ID']).agg(
    # Discounts aggregation
    ORDER_TOTAL_ORGANIC_DISCOUNT=('UNIT_TOTAL_ORGANIC_DISCOUNT', 'sum'),
    ORDER_TOTAL_ORGANIC_USED_DISCOUNT=('UNIT_TOTAL_ORGANIC_DISCOUNT', lambda x: int(x.sum() > 0)),
    ORDER_TOTAL_EXTRA_DISCOUNT=('UNIT_TOTAL_EXTRA_DISCOUNT','sum'),
    ORDER_TOTAL_EXTRA_USED_DISCOUNT=('UNIT_TOTAL_EXTRA_DISCOUNT', lambda x: int(x.sum() > 0)),
    ORDER_TOTAL_DISCOUNT=('UNIT_TOTAL_DISCOUNTS', 'sum'),
    ORDER_TOTAL_USED_DISCOUNT=('UNIT_TOTAL_DISCOUNTS', lambda x: int(x.sum() > 0)),
    # SKU metrics
    ORDER_UNIT_QUANTITY=('UNIT_QUANTITY', 'sum'),
    ORDER_IS_BULK=('UNIT_QUANTITY', lambda x: int(x.sum() > 2)),
    ORDER_IS_BUNDLE=('SKU_IS_BUNDLE', lambda x: (x == 1).any()),
    # OTHER METRICS
    STORE=('STORE_NAME', lambda x: x.unique()[0]),
    STORE_TYPE=('PREFIX', lambda x: x.unique()[0]),
    PAYMENT_METHOD=('PAYMENT_METHOD', lambda x: x.unique()[0])
).reset_index()

# Step 2: Aggregate per CUSTOMER_ID from per-order metrics
behavioral_df = order_level.groupby('CUSTOMER_ID').agg(
    # Discounts aggregatins per customer
    AVG_ORGANIC_DISCOUNT_PER_ORDER=('ORDER_TOTAL_ORGANIC_DISCOUNT', 'mean'),
    USED_ORGANIC_DISCOUNT_RATIO=('ORDER_TOTAL_ORGANIC_USED_DISCOUNT', 'mean'),
    AVG_EXTRA_DISCOUNT_PER_ORDER=('ORDER_TOTAL_EXTRA_DISCOUNT', 'mean'),
    USED_EXTRA_DISCOUNT_RATIO=('ORDER_TOTAL_EXTRA_USED_DISCOUNT', 'mean'),
    AVG_TOTAL_DISCOUNT_PER_ORDER=('ORDER_TOTAL_DISCOUNT', 'mean'),
    USED_TOTAL_DISCOUNT_RATIO=('ORDER_TOTAL_USED_DISCOUNT', 'mean'),
    # SKU metrics per customer
    AVG_UNIT_QUANTITY_PER_ORDER=('ORDER_UNIT_QUANTITY', 'mean'),
    IS_BULK_BUYER=('ORDER_IS_BULK', 'mean'),
    BUNDLE_PURCHASE_RATIO=('ORDER_IS_BUNDLE', 'mean'),
    # OTHER METRICS
    FAVOURITE_STORE=('STORE', lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown'),
    FAVOURITE_STORE_TYPE=('STORE_TYPE', lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown'),
    FAVOURITE_PAYMENT_METHOD=('PAYMENT_METHOD', lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
).reset_index()

# Step 3: Add category diversity and favorite category at SKU level
category_features = valid_orders.groupby('CUSTOMER_ID').agg(
    CATEGORY_DIVERSITY=('SKU_CATEGORY', pd.Series.nunique),
    FAVORITE_CATEGORY=('SKU_CATEGORY', lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown'),
    SUB_CATEGORY_DIVERSITY=('SKU_SUBCATEGORY', pd.Series.nunique),
    FAVORITE_SUB_CATEGORY=('SKU_SUBCATEGORY', lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown'),
    # FAVORITE_CONTAINTER
).reset_index()

behavioral_df = behavioral_df.merge(category_features, on='CUSTOMER_ID', how='left')

In [45]:
rfm_df = pd.merge(rfm_df, behavioral_df, on='CUSTOMER_ID', how='left')

## OPERATIONAL FEATURES


In [46]:
operation_details['WEEKDAY'] = operation_details['ORDER_DATE'].dt.day_name()
operation_details['HOUR'] = operation_details['ORDER_TIME'].dt.hour

In [47]:
valid_orders_operations = operation_details[operation_details['ORDER_STATE_FINAL']=='Valid']

In [48]:
# Calculate operational features per CUSTOMER_ID
operational_features = valid_orders_operations.groupby('CUSTOMER_ID').agg(
    AVG_DELIVERY_TIME=('DELIVERY_TIME', 'mean'),
    SLA_VIOLATION_RATE=('WITHIN_SLA', lambda x: (x == 0).sum() / len(x)),
    AVG_APPROVAL_TIME=('APPROVAL_TIME', 'mean'),
    AVG_TOTAL_DISTANCE=('TOTAL_DISTANCE', 'mean'),
    FAVORITE_WEEKDAY=('WEEKDAY', lambda x: x.mode()[0] if not x.mode().empty else 'Unknown'),
    FAVORITE_HOUR=('HOUR', lambda x: x.mode()[0] if not x.mode().empty else -1)
).reset_index()

In [49]:
extended_rfm_df = rfm_df.merge(operational_features, on='CUSTOMER_ID', how='left')

In [50]:
extended_rfm_df.to_parquet("../csv_export/RFM.parquet", index=False)

In [51]:
extended_rfm_df.head()

,CUSTOMER_ID,Recency,Frequency,Monetary,Total_Units,Total_Organic_Discount,Total_Extra_Discount,Total_Discounts,AVG_ORGANIC_DISCOUNT_PER_ORDER,USED_ORGANIC_DISCOUNT_RATIO,...,CATEGORY_DIVERSITY,FAVORITE_CATEGORY,SUB_CATEGORY_DIVERSITY,FAVORITE_SUB_CATEGORY,AVG_DELIVERY_TIME,SLA_VIOLATION_RATE,AVG_APPROVAL_TIME,AVG_TOTAL_DISTANCE,FAVORITE_WEEKDAY,FAVORITE_HOUR
0,00002b303bd5e3641c7deb26f7a66bd83b202c4da03b08...,487,1,61.3900,1,19.11,0.0000,19.1100,19.110000,1.000000,...,1,Beer,1,Beer,31.000000,0.000000,0.000000,0.008983,Thursday,12
1,00008655ece97d61c76ec17e483ac6fd7813e5a15c498a...,3,5,670.8799,6,86.53,41.7701,128.3001,17.306000,1.000000,...,2,Beer,2,Beer,29.800000,0.000000,0.000000,0.064991,Monday,11
2,0000bc0f1d0509b07a0472ce0be544d25e1d90a83dcf3a...,109,7,610.5100,16,2.59,107.0000,109.5900,0.370000,0.142857,...,2,Beer,2,Beer,23.000000,0.000000,1.285714,0.045375,Saturday,13
3,0000d61a21fef1a1a0be35e5ad84acaad0af9039d7096a...,491,1,30.3882,1,4.00,0.0118,4.0118,4.000000,1.000000,...,1,Beer,1,Beer,9.000000,0.000000,0.000000,0.007809,Sunday,13
4,00013de18abb79cce6126077cdd1911d5e02141dc1b5d9...,44,27,2187.3447,55,320.90,85.1053,406.0053,11.885185,0.703704,...,5,Beer,7,Beer,21.555556,0.037037,1.074074,0.121917,Saturday,0
